# Исследование надежности заемщиков.

<div class="alert alert-info">
<h2> Комментарий студента</h2>

Поставленная задача:
Необходимо определить, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.
Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

Описание данных датасета:

1. children — количество детей в семье
2. days_employed — общий трудовой стаж в днях
3. dob_years — возраст клиента в годах
4. education — уровень образования клиента
5. education_id — идентификатор уровня образования
6. family_status — семейное положение
7. family_status_id — идентификатор семейного положения
8. gender — пол клиента
9. income_type — тип занятости
10. debt — имел ли задолженность по возврату кредитов
11. total_income — ежемесячный доход
12. purpose — цель получения кредита
</div>

### Шаг 1. Обзор данных
Импортируем библиотеку 'pandas' и присвоим переменной 'data' датафрейм. Выведем на экран первые пять строчек таблицы.

In [1]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Посмотрим сводные данные о датафрейме 'data'.

In [2]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


Колонки 'days_employed' и 'total_income' содержат значения 'Nan' вместо чисел. Просмотрим эти колонки и удостоверимся в этом.

In [3]:
data['days_employed'].isna().sort_values()

0        False
13951    False
13950    False
13949    False
13948    False
         ...  
18354     True
18355     True
18356     True
2813      True
10762     True
Name: days_employed, Length: 21525, dtype: bool

In [4]:
data['total_income'].isna().sort_values()

0        False
13951    False
13950    False
13949    False
13948    False
         ...  
18354     True
18355     True
18356     True
2813      True
10762     True
Name: total_income, Length: 21525, dtype: bool

Столбец 'children' седержит артефакты, в виде значения "-1", "20".

In [5]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

Столбец 'days_employed' содержит значения которые соответствуют трудовому стажу юолее 900 лет. Выведу на экран только первую строку этого списка.

In [6]:
(data['days_employed'][data['days_employed'] > 900 * 365]).head(1)

4    340266.072047
Name: days_employed, dtype: float64

Столбец 'dob_years' содержит значения "0".

In [7]:
data['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

Столбец 'education' содержит неявные дубликаты.

In [8]:
data['education'].sort_values().unique()

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

Исходный датафрейм содержит ряд ошибок:
1. Колонки 'days_employed' и 'total_income' содержат значения 'Nan' вместо чисел.
2. Столбец 'days_employed' содержит отричательные значения.
3. Столбец 'children' седержит артефакты, в виде значения "-1", "20".
4. Столбец 'days_employed' содержит значения которые соответствуют трудовому стажу юолее 900 лет.
5. Столбец 'dob_years' содержит значения "0".
6. Столбец 'education' содержит неявные дубликаты.

### Шаг 2.1 Проверка данных на аномалии и исправления.

#### Столбец 'children'.
Проверим уникальные значения столбца 'children'. 

In [9]:
data['children'].sort_values().unique()

array([-1,  0,  1,  2,  3,  4,  5, 20])

Видно что кол-во детей находится в пределах "0 - 5" шт., а значения "-1" и "20" это явные ошибки. Делаем предположение, что это опечатки и заменяем "-1" на "1", "20" на "2". Посмотрим на строки с кол-вом детей 20шт. Никакой  акономерности не прослеживается, еще раз убеждаемся что это "баг".

In [10]:
data[data['children'] == 20]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
606,20,-880.221113,21,среднее,1,женат / замужем,0,M,компаньон,0,145334.865002,покупка жилья
720,20,-855.595512,44,среднее,1,женат / замужем,0,F,компаньон,0,112998.738649,покупка недвижимости
1074,20,-3310.411598,56,среднее,1,женат / замужем,0,F,сотрудник,1,229518.537004,получение образования
2510,20,-2714.161249,59,высшее,0,вдовец / вдова,2,F,сотрудник,0,264474.835577,операции с коммерческой недвижимостью
2941,20,-2161.591519,0,среднее,1,женат / замужем,0,F,сотрудник,0,199739.941398,на покупку автомобиля
...,...,...,...,...,...,...,...,...,...,...,...,...
21008,20,-1240.257910,40,среднее,1,женат / замужем,0,F,сотрудник,1,133524.010303,свой автомобиль
21325,20,-601.174883,37,среднее,1,женат / замужем,0,F,компаньон,0,102986.065978,профильное образование
21390,20,NaN,53,среднее,1,женат / замужем,0,M,компаньон,0,NaN,покупка жилой недвижимости
21404,20,-494.788448,52,среднее,1,женат / замужем,0,M,компаньон,0,156629.683642,операции со своей недвижимостью


Заменим все значения "-1" и "20" в датафрейме в столбце 'children' на "1" и "2" соответственно.

In [11]:
data['children'] = data['children'].replace(20, 2)
data['children'] = data['children'].replace(-1, 1)

Проверим результат:

In [12]:
data['children'].sort_values().unique()

array([0, 1, 2, 3, 4, 5])

#### Столбец 'dob_years'.

Проверим колонку 'dob_years' (возраст клиента в годах) на уникальные значения. 

In [13]:
data['dob_years'].sort_values().unique()

array([ 0, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75])

В выведенных данных содержится значение "0". Посмотрим на строки с этим значением в колонке 'dob_years'.

In [14]:
(data[data['dob_years'] == 0]).head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
99,0,346541.618895,0,Среднее,1,женат / замужем,0,F,пенсионер,0,71291.522491,автомобиль
149,0,-2664.273168,0,среднее,1,в разводе,3,F,сотрудник,0,70176.435951,операции с жильем
270,3,-1872.663186,0,среднее,1,женат / замужем,0,F,сотрудник,0,102166.458894,ремонт жилью
578,0,397856.565013,0,среднее,1,женат / замужем,0,F,пенсионер,0,97620.687042,строительство собственной недвижимости
1040,0,-1158.029561,0,высшее,0,в разводе,3,F,компаньон,0,303994.134987,свой автомобиль
1149,0,-934.654854,0,среднее,1,женат / замужем,0,F,компаньон,0,201852.430096,покупка недвижимости
1175,0,370879.508002,0,среднее,1,женат / замужем,0,F,пенсионер,0,313949.845188,получение дополнительного образования
1386,0,-5043.219890,0,высшее,0,женат / замужем,0,M,госслужащий,0,240523.618071,сделка с автомобилем
1890,0,NaN,0,высшее,0,Не женат / не замужем,4,F,сотрудник,0,NaN,жилье
1898,0,370144.537021,0,среднее,1,вдовец / вдова,2,F,пенсионер,0,127400.268338,на покупку автомобиля


In [15]:
print('Кол-во строк в которых возраст клиента равен "нулю" составляет:',
      len(data[data['dob_years'] == 0]), 'шт.', '-', 
      round(len(data[data['dob_years'] == 0]) / len(data['dob_years']) *100,
            2), '% от общего кол-ва строк ДФ')

Кол-во строк в которых возраст клиента равен "нулю" составляет: 101 шт. - 0.47 % от общего кол-ва строк ДФ


Кол-во строк меньше 0,5%. Это относительно малое значение и можно удалить эти строки т.к. не получится востановить иинформацию по ним.

In [16]:
data = data[data['dob_years'] != 0]

Проверим кол-во строк в которых возраст клиента равен "0":

In [17]:
print('Кол-во строк в которых возраст клиента равен "нулю":',
      len(data[data['dob_years'] == 0]), 'шт.')

Кол-во строк в которых возраст клиента равен "нулю": 0 шт.


#### Столбец 'days_employed'

Заменим все отрицательные значения в столбце 'days_employed' на положительные.

In [18]:
data['days_employed'] = data['days_employed'].abs()

In [19]:
print('Кол-во отричательных значений в столбце "days_employed":',data['days_employed'][data['days_employed'] < 0].count(),'шт.')

Кол-во отричательных значений в столбце "days_employed": 0 шт.


Переведем в года занчения столбца 'days_employed' с точностью до десятых. Т.к. далее сортировка будет выполняться по возрастным группам, считаю точность определения лет стажа достаточной.

In [20]:
data['years_employed'] = round(data['days_employed'] / 365 , 1)
display(data)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
0,1,8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья,23.1
1,1,4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля,11.0
2,0,5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья,15.4
3,3,4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование,11.3
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу,932.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21520,1,4529.316663,43,среднее,1,гражданский брак,1,F,компаньон,0,224791.862382,операции с жильем,12.4
21521,0,343937.404131,67,среднее,1,женат / замужем,0,F,пенсионер,0,155999.806512,сделка с автомобилем,942.3
21522,1,2113.346888,38,среднее,1,гражданский брак,1,M,сотрудник,1,89672.561153,недвижимость,5.8
21523,3,3112.481705,38,среднее,1,женат / замужем,0,M,сотрудник,1,244093.050500,на покупку своего автомобиля,8.5


In [21]:
years_employed = 51
employed_until = data[data['years_employed'] <= years_employed]
employed_above = data[data['years_employed'] >= years_employed]
print('Стаж', years_employed,'год и менее:',
      len(employed_until['years_employed']),'человека','-',
     round(len(employed_until['years_employed']) / 
           len(data['years_employed']) * 100,1), '%')
print('Стаж более', years_employed, 'года:',
      len(employed_above['years_employed']),'человек', '-',
     round(len(employed_above['years_employed']) / 
           len(data['years_employed']) * 100,1), '%')

Стаж 51 год и менее: 15832 человека - 73.9 %
Стаж более 51 года: 3428 человек - 16.0 %


Стаж в 51 год это пороговое значение, которое отделяет нереальные показатели от реально возможных. Выведем на экран список уникальных знвчений колонки 'eyers_employed'.

In [22]:
employed_above['years_employed'].sort_values().unique()

array([ 900.6,  900.7,  900.8, ..., 1100.5, 1100.6, 1100.7])

Посмотрим какие категории людей попали в список с нераельным стажем работы.

In [23]:
employed_above['income_type'].unique()

array(['пенсионер', 'безработный'], dtype=object)

Доля строк велика и удалять их нельзя. Пока оставим эти строки, возможно в дальнейшем заменим их на медианные значения по "стажу работы".

#### Столбец 'gender'.

Посмотрим уникальные значения столбца 'gender'.

In [24]:
print(data['gender'].sort_values().unique())

['F' 'M' 'XNA']


В выборке по 'полу' присутствует одно значение, которое не поддается описанию, оставим пока это без изменений. Выведем эту строчку и проанализируем ее.

In [25]:
data[data['gender'] == 'XNA']

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,years_employed
10701,0,2358.600502,24,неоконченное высшее,2,гражданский брак,1,XNA,компаньон,0,203905.157261,покупка недвижимости,6.5


Больше похоже на ошибку системы при записи данных. Если в дальнейшем придется фильтровать по полу, то удалим эту строку.

### Шаг 2.2 Удаление дубликатов.

#### Столбец 'education'

Выведем на экран уникальные значения колонки 'education'

In [26]:
data['education'].sort_values().unique()

array(['ВЫСШЕЕ', 'Высшее', 'НАЧАЛЬНОЕ', 'НЕОКОНЧЕННОЕ ВЫСШЕЕ',
       'Начальное', 'Неоконченное высшее', 'СРЕДНЕЕ', 'Среднее',
       'УЧЕНАЯ СТЕПЕНЬ', 'Ученая степень', 'высшее', 'начальное',
       'неоконченное высшее', 'среднее', 'ученая степень'], dtype=object)

Напишем функцию, которая заменит все неявные дубликаты в колонке 'education'

In [27]:
def replace_wrong_education(wrong_education):
    for item in wrong_education:
        if item == 'ВЫСШЕЕ' or item == 'Высшее':
            correct_education = 'высшее'
            data['education'] = data['education'].replace(item, correct_education)
        if item == 'НАЧАЛЬНОЕ':
            correct_education = 'начальное'
            data['education'] = data['education'].replace(item, correct_education)
        if item == 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' or item == 'Неоконченное высшее':
            correct_education = 'неоконченное высшее'
            data['education'] = data['education'].replace(item, correct_education)
        if item == 'Начальное':
            correct_education = 'начальное'
            data['education'] = data['education'].replace(item, correct_education)
        if item == 'СРЕДНЕЕ' or item == 'Среднее':
            correct_education = 'среднее'
            data['education'] = data['education'].replace(item, correct_education)
        if item == 'УЧЕНАЯ СТЕПЕНЬ' or item == 'Ученая степень':
            correct_education = 'ученая степень'
            data['education'] = data['education'].replace(item, correct_education)

In [28]:
wrong_education = data['education']
replace_wrong_education(wrong_education)
data['education'].sort_values().unique()

array(['высшее', 'начальное', 'неоконченное высшее', 'среднее',
       'ученая степень'], dtype=object)

Неявные дубликаты отсутствуют.

#### Столбец 'family_status'

Посмотрим уникальные значения столбца 'family_status'.

In [29]:
print(data['family_status'].sort_values().unique())

['Не женат / не замужем' 'в разводе' 'вдовец / вдова' 'гражданский брак'
 'женат / замужем']


Приведем к красивому виду значения в колоннке 'family_status'.

In [30]:
data['family_status'] = data['family_status'].replace('Не женат / не замужем', 
                                                      'не женат / не замужем')

Посмотрим уникальные значения столбца 'family_status'.

In [31]:
print(data['family_status'].sort_values().unique())

['в разводе' 'вдовец / вдова' 'гражданский брак' 'женат / замужем'
 'не женат / не замужем']


Посмотрим уникальные значения столбца 'family_status_id'.

In [32]:
print(data['family_status_id'].sort_values().unique())

[0 1 2 3 4]


In [33]:
print('Кол-во строк дубликатов в таблице data:',data.duplicated().sum()) # подсчёт явных дубликатов

Кол-во строк дубликатов в таблице data: 71


In [34]:
#Применим метод "drop_duplicates" для удаления явных дубликатов.
data.drop_duplicates()
#Сбросим индексы в датасете.
data = data.dropna().reset_index(drop=True)
print('Кол-во строк дубликатов после очистки данных:',data.duplicated().sum()) # подсчёт явных дубликатов

Кол-во строк дубликатов после очистки данных: 0


### Шаг 2.3. Изменение типов данных.

#### Столбец 'total_income'.

Проверим колонку на наличие отрицательных значений.

In [35]:
print('Кол-во отрицательных значений в колонке "total_income":',
      (data['total_income'][data['total_income'] < 0].count()),'шт.')

Кол-во отрицательных значений в колонке "total_income": 0 шт.


Переведем значения столбца 'total_income' к целочисленному виду.

In [36]:
print('Исходный тип данных колонки "total_income":',data['total_income'].dtypes)

Исходный тип данных колонки "total_income": float64


Для начала посчитаем кол-во строк со значением "0" и 'Nan'.

In [37]:
print('Кол-во строк со значением "Nan":',data['total_income'].isna().sum(),
      'шт.')
print('Кол-во строк со значением "0" в колонке "total_income":',
      (data['total_income'][data['total_income'] == 0].count()),'шт.')

Кол-во строк со значением "Nan": 0 шт.
Кол-во строк со значением "0" в колонке "total_income": 0 шт.


In [38]:
data['total_income'] = data['total_income'].fillna(0)
print('Кол-во строк со значением "0":',
      data[data['total_income'] == 0]['total_income'].count(), 'шт.')

Кол-во строк со значением "0": 0 шт.


Произведем замену типа данных колонки 'total_income'.

In [39]:
data['total_income'] = data['total_income'].astype('int64')

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19260 entries, 0 to 19259
Data columns (total 13 columns):
children            19260 non-null int64
days_employed       19260 non-null float64
dob_years           19260 non-null int64
education           19260 non-null object
education_id        19260 non-null int64
family_status       19260 non-null object
family_status_id    19260 non-null int64
gender              19260 non-null object
income_type         19260 non-null object
debt                19260 non-null int64
total_income        19260 non-null int64
purpose             19260 non-null object
years_employed      19260 non-null float64
dtypes: float64(2), int64(6), object(5)
memory usage: 1.9+ MB


В обоих столбцах доля пропущенных значений составляет 10%. Это достаточно болщой процент и удаление такого кол-ва данных повлияет на результат иследования. Поэтому заменим пропуски на медианное значение по столбцу.

### Шаг 2.4. Заполнение пропусков.

In [41]:
#Посчитаем кол-во строк со значением 'Nan' в столбце 'days_employed'.
print('Кол-во строк со значением "Nan" в столбце "days_employed":',
      len(data[data['days_employed'].isna()]), 'шт.')

#Определим долю этих строк в общем кол-ве строк серии 'days_employed'.
print('Процент испорченных строк:', 
        round(len(data[data['days_employed'].isna()]) / len(data['dob_years']) *
        100,2), '%')

Кол-во строк со значением "Nan" в столбце "days_employed": 0 шт.
Процент испорченных строк: 0.0 %


In [42]:
#Посчитаем кол-во строк со значением 'Nan' в столбце 'years_employed'.
print('Кол-во строк со значением "Nan" в столбце "years_employed":',
      len(data[data['years_employed'].isna()]), 'шт.')

#Определим долю этих строк в общем кол-ве строк серии 'days_employed'.
print('Процент испорченных строк:', 
        round(len(data[data['years_employed'].isna()]) / len(data['dob_years']) *
        100,2), '%')

Кол-во строк со значением "Nan" в столбце "years_employed": 0 шт.
Процент испорченных строк: 0.0 %


In [43]:
#Т.к. в столбце 'total_income' значения 'Nan' заменены на '0',
#посчитаем кол-во строк со значением '0' в столбце 'total_income'.
print('Кол-во строк со значением "0" в столбце "total_income":',
      len(data[data['total_income'] == 0]), 'шт.')

#Определим долю этих строк в общем кол-ве строк серии 'total_income'.
print('Процент испорченных строк:', round(len(data[data['total_income'] == 0]) / 
                                          len(data['dob_years']) * 100,2), '%')

Кол-во строк со значением "0" в столбце "total_income": 0 шт.
Процент испорченных строк: 0.0 %


В обоих столбцах доля пропущенных значений составляет 10%. Это достаточно болщой процент и удаление такого кол-ва данных повлияет на результат иследования. Поэтому заменим пропуски на медианное значение по столбцу.

In [44]:
#Посчитаем медиану по столбцам 'days_employed', 'total_income', 'years_employed'
mediana_days_employed = round(data['days_employed'].median(),2)
mediana_years_employed = round(data['years_employed'].median(),2)
mediana_total_income = round(data['total_income'].median(),2)
print('Значение медианы по столбцу "days_employed" составляет:',mediana_days_employed)
print('Значение медианы по столбцу "years_employed" составляет:',mediana_years_employed)
print('Значение медианы по столбцу "total_income" составляет:', mediana_total_income)

Значение медианы по столбцу "days_employed" составляет: 2197.36
Значение медианы по столбцу "years_employed" составляет: 6.0
Значение медианы по столбцу "total_income" составляет: 145011.0


In [45]:
#Замена пропусков медианным значением
data['days_employed'] = data['days_employed'].fillna(value=mediana_days_employed)
data['years_employed'] = data['years_employed'].fillna(value=mediana_years_employed)
data['total_income'] = data['total_income'].replace(0,mediana_total_income)

### Шаг 2.5. Формирование дополнительных датафреймов словарей, декомпозиция исходного датафрейма.

Создание датафреймов - словарей 'education_id' и 'family_status_id'.

In [46]:
education_id = pd.DataFrame(data=data, columns=['education_id', 'education'])
family_status_id = pd.DataFrame(data=data, columns=['family_status_id', 'family_status'])

Удаление столбцов 'education', 'family_status' из исходного датафрейма.

In [47]:
data = data.drop(['education', 'family_status'], axis=1)

In [49]:
# подсчёт явных дубликатов
print('Кол-во строк дубликатов в "education_id":',education_id.duplicated().sum())

#Применим метод "drop_duplicates" для удаления явных дубликатов.
education_id = education_id.drop_duplicates()

#Сбросим индексы в датасете.
education_id = education_id.dropna().reset_index(drop=True)
print('Кол-во строк дубликатов после очистки данных в "education_id":',education_id.duplicated().sum()) # подсчёт явных дубликатов
education_id

Кол-во строк дубликатов в "education_id": 19255
Кол-во строк дубликатов после очистки данных в "education_id": 0


,education_id,education
0,0,высшее
1,1,среднее
2,2,неоконченное высшее
3,3,начальное
4,4,ученая степень


In [50]:
# подсчёт явных дубликатов
print('Кол-во строк дубликатов в "family_status_id":',family_status_id.duplicated().sum())

#Применим метод "drop_duplicates" для удаления явных дубликатов.
family_status_id = family_status_id.drop_duplicates()

#Сбросим индексы в датасете.
family_status_id = family_status_id.dropna().reset_index(drop=True)
print('Кол-во строк дубликатов после очистки данных в "family_status_id":',family_status_id.duplicated().sum()) # подсчёт явных дубликатов
family_status_id

Кол-во строк дубликатов в "family_status_id": 19255
Кол-во строк дубликатов после очистки данных в "family_status_id": 0


,family_status_id,family_status
0,0,женат / замужем
1,1,гражданский брак
2,2,вдовец / вдова
3,3,в разводе
4,4,не женат / не замужем


### Шаг 2.6. Категоризация дохода.

#### Напишем функцию, которая создает новый столбец 'total_income_category' и заполняет категорями в зависимости от значений в столбце 'total_income'.

In [51]:
temp_df = data
income = data['total_income']

def income_category(total_income):
    #total_income = df[df['total_income']]
    if 0 <= total_income <= 30000:
        return 'E'
    if 30001 <= total_income <= 50000:
        return 'D'
    if 50001 <= total_income <= 200000:
        return 'C'
    if 200001 <= total_income <= 1000000:
        return 'B'
    if total_income >= 1000001:
        return 'A'

data['total_income_category'] = temp_df['total_income'].apply(income_category)
data.groupby('total_income_category')['gender'].count()

total_income_category
A       25
B     5015
C    13850
D      348
E       22
Name: gender, dtype: int64

### Шаг 2.7. Категоризация целей кредита.

In [52]:
#Посмотрим уникальные значения столбца 'purpose'.
print(data['purpose'].sort_values().unique())

['автомобили' 'автомобиль' 'высшее образование'
 'дополнительное образование' 'жилье' 'заняться высшим образованием'
 'заняться образованием' 'на покупку автомобиля'
 'на покупку подержанного автомобиля' 'на покупку своего автомобиля'
 'на проведение свадьбы' 'недвижимость' 'образование' 'операции с жильем'
 'операции с коммерческой недвижимостью' 'операции с недвижимостью'
 'операции со своей недвижимостью' 'покупка жилой недвижимости'
 'покупка жилья' 'покупка жилья для сдачи' 'покупка жилья для семьи'
 'покупка коммерческой недвижимости' 'покупка недвижимости'
 'покупка своего жилья' 'получение высшего образования'
 'получение дополнительного образования' 'получение образования'
 'приобретение автомобиля' 'профильное образование' 'ремонт жилью'
 'свадьба' 'свой автомобиль' 'сделка с автомобилем'
 'сделка с подержанным автомобилем' 'строительство жилой недвижимости'
 'строительство недвижимости' 'строительство собственной недвижимости'
 'сыграть свадьбу']


#### Напишем функцию, которая создает новый столбец 'purpose_category' и заполняет категорями в зависимости от значений в столбце 'purpose'.

In [53]:
temp_df = data
purpose = data['purpose']

def purpose_category(purpose):
    if 'жиль' in purpose:
        return 'операции с недвижимостью'
    if 'недвижимост' in purpose:
        return 'операции с недвижимостью'
    if 'автомобил' in purpose:
        return 'операции с автомобилем'
    if 'свадьб' in purpose:
        return 'проведение свадьбы'
    if 'образовани' in purpose:
        return 'получение образования'
    
data['purpose_category'] = temp_df['purpose'].apply(purpose_category)
data.groupby('purpose_category')['purpose'].count()

purpose_category
операции с автомобилем      3875
операции с недвижимостью    9719
получение образования       3579
проведение свадьбы          2087
Name: purpose, dtype: int64

### Ответы на вопросы.

##### Вопрос 1: Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [54]:
data.groupby('children')['debt'].count()

children
0    12649
1     4372
2     1904
3      293
4       34
5        8
Name: debt, dtype: int64

In [55]:
print('Кол-во заемщиков без детей не имевших задолжности:',
      data['children'][(data['children'] == 0) & (data['debt'] == 0)].count(),
      'человек',
      '-',
      round(data['children'][(data['children'] == 0) & (data['debt'] == 0)].count() 
                                                        / data['children'][data['children'] == 0].count() * 100, 2),
      '%')

print('Кол-во заемщиков без детей имевших задолжности:',
      data['children'][(data['children'] == 0) & (data['debt'] == 1)].count(),
      'человек',
      '-',
      round(data['children'][(data['children'] == 0) & (data['debt'] == 1)].count() 
                                                        / data['children'][data['children'] == 0].count() * 100, 2),
      '%')

print('------------------------------------------------------------------------')

print('Кол-во заемщиков с одним ребенком не имевших задолжности:',
      data['children'][(data['children'] == 1) & (data['debt'] == 0)].count(),
      'человек',
      '-',
      round(data['children'][(data['children'] == 1) & (data['debt'] == 0)].count() 
                                                        / data['children'][data['children'] == 1].count() * 100, 2),
      '%')

print('Кол-во заемщиков с одним ребенком имевших задолжности:',
      data['children'][(data['children'] == 1) & (data['debt'] == 1)].count(),
      'человек',
      '-',
      round(data['children'][(data['children'] == 1) & (data['debt'] == 1)].count() 
                                                        / data['children'][data['children'] == 1].count() * 100, 2),
      '%')

print('------------------------------------------------------------------------')

print('Кол-во заемщиков с двумя детьми не имевших задолжности:',
      data['children'][(data['children'] == 2) & (data['debt'] == 0)].count(),
      'человек',
      '-',
      round(data['children'][(data['children'] == 2) & (data['debt'] == 0)].count() 
                                                        / data['children'][data['children'] == 2].count() * 100, 2),
      '%')

print('Кол-во заемщиков с двумя детьми имевших задолжности:',
      data['children'][(data['children'] == 2) & (data['debt'] == 1)].count(),
      'человек',
      '-',
      round(data['children'][(data['children'] == 2) & (data['debt'] == 1)].count() 
                                                        / data['children'][data['children'] == 2].count() * 100, 2),
      '%')

print('------------------------------------------------------------------------')

print('Кол-во заемщиков с тремя детьми не имевших задолжности:',
      data['children'][(data['children'] == 3) & (data['debt'] == 0)].count(),
      'человек',
      '-',
      round(data['children'][(data['children'] == 3) & (data['debt'] == 0)].count() 
                                                        / data['children'][data['children'] == 3].count() * 100, 2),
      '%')

print('Кол-во заемщиков с тремя детьми имевших задолжности:',
      data['children'][(data['children'] == 3) & (data['debt'] == 1)].count(),
      'человек',
      '-',
      round(data['children'][(data['children'] == 3) & (data['debt'] == 1)].count() 
                                                        / data['children'][data['children'] == 3].count() * 100, 2),
      '%')

print('------------------------------------------------------------------------')

print('Кол-во заемщиков с четырьмя детьми не имевших задолжности:',
      data['children'][(data['children'] == 4) & (data['debt'] == 0)].count(),
      'человек',
      '-',
      round(data['children'][(data['children'] == 4) & (data['debt'] == 0)].count() 
                                                        / data['children'][data['children'] == 4].count() * 100, 2),
      '%')

print('Кол-во заемщиков с четырьмя детьми имевших задолжности:',
      data['children'][(data['children'] == 4) & (data['debt'] == 1)].count(),
      'человек',
      '-',
      round(data['children'][(data['children'] == 4) & (data['debt'] == 1)].count() 
                                                        / data['children'][data['children'] == 4].count() * 100, 2),
      '%')

print('------------------------------------------------------------------------')

print('Кол-во заемщиков с пятью детьми не имевших задолжности:',
      data['children'][(data['children'] == 5) & (data['debt'] == 0)].count(),
      'человек',
      '-',
      round(data['children'][(data['children'] == 5) & (data['debt'] == 0)].count() 
                                                        / data['children'][data['children'] == 5].count() * 100, 2),
      '%')

print('Кол-во заемщиков с пятью детьми имевших задолжности:',
      data['children'][(data['children'] == 5) & (data['debt'] == 1)].count(),
      'человек',
      '-',
      round(data['children'][(data['children'] == 5) & (data['debt'] == 1)].count() 
                                                        / data['children'][data['children'] == 5].count() * 100, 2),
      '%')


Кол-во заемщиков без детей не имевших задолжности: 11702 человек - 92.51 %
Кол-во заемщиков без детей имевших задолжности: 947 человек - 7.49 %
------------------------------------------------------------------------
Кол-во заемщиков с одним ребенком не имевших задолжности: 3966 человек - 90.71 %
Кол-во заемщиков с одним ребенком имевших задолжности: 406 человек - 9.29 %
------------------------------------------------------------------------
Кол-во заемщиков с двумя детьми не имевших задолжности: 1719 человек - 90.28 %
Кол-во заемщиков с двумя детьми имевших задолжности: 185 человек - 9.72 %
------------------------------------------------------------------------
Кол-во заемщиков с тремя детьми не имевших задолжности: 271 человек - 92.49 %
Кол-во заемщиков с тремя детьми имевших задолжности: 22 человек - 7.51 %
------------------------------------------------------------------------
Кол-во заемщиков с четырьмя детьми не имевших задолжности: 31 человек - 91.18 %
Кол-во заемщиков с четы

##### Вывод 1: Вывод очевиден, количество человек имевших задолжности по выплате кредитов растет с количеством детей. Группы с тремя и более детьми достаточно малочисленны от общей выборки, и эти данные можно не использовать.

##### Вопрос 2: Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [59]:
data['family_status_id'].unique()

array([0, 1, 2, 3, 4])

In [60]:
def family_status_result(status_id=0):
    
    idx_series = family_status_id[family_status_id['family_status_id'] == status_id].index
    idx = idx_series[0]
    family_status = family_status_id.iloc[idx, 1]
    
    total_user = data['family_status_id'][data['family_status_id'] == status_id].count()
    user_wo_debt = data['family_status_id'][(data['family_status_id'] == status_id) & (data['debt'] == 0)].count()
    user_w_debt = data['family_status_id'][(data['family_status_id'] == status_id) & (data['debt'] == 1)].count()
    
    print(f'Кол-во кредиторов с семейным статусом - "{family_status}", не имевших задолжности:', user_wo_debt,
                                                'человек', '-', round(user_wo_debt / total_user * 100,2), '%')
    
    print(f'Кол-во кредиторов с семейным статусом - "{family_status}", имевших задолжности:', user_w_debt,
                                                'человек', '-', round(user_w_debt / total_user * 100,2), '%')

In [61]:
family_status_result(0)

Кол-во кредиторов с семейным статусом - "женат / замужем", не имевших задолжности: 10256 человек - 92.41 %
Кол-во кредиторов с семейным статусом - "женат / замужем", имевших задолжности: 842 человек - 7.59 %


In [62]:
family_status_result(1)

Кол-во кредиторов с семейным статусом - "гражданский брак", не имевших задолжности: 3380 человек - 90.93 %
Кол-во кредиторов с семейным статусом - "гражданский брак", имевших задолжности: 337 человек - 9.07 %


In [63]:
family_status_result(2)

Кол-во кредиторов с семейным статусом - "вдовец / вдова", не имевших задолжности: 806 человек - 93.61 %
Кол-во кредиторов с семейным статусом - "вдовец / вдова", имевших задолжности: 55 человек - 6.39 %


In [64]:
family_status_result(3)

Кол-во кредиторов с семейным статусом - "в разводе", не имевших задолжности: 998 человек - 92.92 %
Кол-во кредиторов с семейным статусом - "в разводе", имевших задолжности: 76 человек - 7.08 %


In [65]:
family_status_result(4)

Кол-во кредиторов с семейным статусом - "не женат / не замужем", не имевших задолжности: 2257 человек - 89.92 %
Кол-во кредиторов с семейным статусом - "не женат / не замужем", имевших задолжности: 253 человек - 10.08 %


In [66]:
#Второй вариант решения данной задачи.
def function_family_status_result():
    for status_id in range(len(data['family_status_id'].unique())):
        idx_series = family_status_id[family_status_id['family_status_id'] == status_id].index
        idx = idx_series[0]
        family_status = family_status_id.iloc[idx, 1]

        total_user = data['family_status_id'][data['family_status_id'] == status_id].count()
        user_wo_debt = data['family_status_id'][(data['family_status_id'] == status_id) & (data['debt'] == 0)].count()
        user_w_debt = data['family_status_id'][(data['family_status_id'] == status_id) & (data['debt'] == 1)].count()

        #print(f'Кол-во кредиторов с семейным статусом - "{family_status}", не имевших задолжности:', user_wo_debt,
                                                    #'человек', '-', round(user_wo_debt / total_user * 100,2), '%')

        print(f'Кол-во кредиторов с семейным статусом - "{family_status}", имевших задолжности:', user_w_debt,
                                                    'человек', '-', round(user_w_debt / total_user * 100,2), '%')
        #print('------------------------------------------------------------------------------------------------')

In [67]:
function_family_status_result()

Кол-во кредиторов с семейным статусом - "женат / замужем", имевших задолжности: 842 человек - 7.59 %
Кол-во кредиторов с семейным статусом - "гражданский брак", имевших задолжности: 337 человек - 9.07 %
Кол-во кредиторов с семейным статусом - "вдовец / вдова", имевших задолжности: 55 человек - 6.39 %
Кол-во кредиторов с семейным статусом - "в разводе", имевших задолжности: 76 человек - 7.08 %
Кол-во кредиторов с семейным статусом - "не женат / не замужем", имевших задолжности: 253 человек - 10.08 %


##### Вывод 2: Семейное положение влияет на способность людей отдавать кредит в срок. Люди находящиеся в категориях - "гражданский брак" и "не женат / не замужем" чаще других тмеют проблемы с возвратом кредита в срок.

##### Вопрос 3: Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [68]:
import numpy as np
total_income_category = data['total_income_category'].unique()
total_income_category = np.sort(total_income_category)

def function_total_income():
    for income_category in total_income_category:
        
        massiv = np.array(total_income_category)
        index = massiv.tolist().index(income_category)
        
        category = total_income_category[index]

        total_user = data['total_income_category'][data['total_income_category'] == income_category].count()
        user_wo_debt = data['total_income_category'][(data['total_income_category'] == income_category) 
                                                     & (data['debt'] == 0)].count()
        user_w_debt = data['total_income_category'][(data['total_income_category'] == income_category) 
                                                    & (data['debt'] == 1)].count()

        #print(f'Кол-во кредиторов с уровнем дохода категории - "{income_category}", не имевших задолжности:', user_wo_debt,
                                                    #'человек', '-', round(user_wo_debt / total_user * 100,2), '%')

        print(f'Кол-во кредиторов с уровнем дохода категории - "{income_category}", имевших задолжности:', user_w_debt,
                                                    'человек', '-', round(user_w_debt / total_user * 100,2), '%')
        #print('------------------------------------------------------------------------------------------------')

In [69]:
function_total_income()

Кол-во кредиторов с уровнем дохода категории - "A", имевших задолжности: 2 человек - 8.0 %
Кол-во кредиторов с уровнем дохода категории - "B", имевших задолжности: 355 человек - 7.08 %
Кол-во кредиторов с уровнем дохода категории - "C", имевших задолжности: 1183 человек - 8.54 %
Кол-во кредиторов с уровнем дохода категории - "D", имевших задолжности: 21 человек - 6.03 %
Кол-во кредиторов с уровнем дохода категории - "E", имевших задолжности: 2 человек - 9.09 %


##### Вывод 3: Линейной закономергости не прослеживается в результатах иследования. В виду того, что категории "A, D,E" достаточно малочисленны, можно выделить проблемную категорию с уровнем дохода - категория "С".

##### Вопрос 4: Как разные цели кредита влияют на его возврат в срок?


In [70]:
import numpy as np
purpose_category = data['purpose_category'].unique()
purpose_category = np.sort(purpose_category)

def function_purpose_category():
    for purpose in purpose_category:
        
        massiv = np.array(purpose_category)
        index = massiv.tolist().index(purpose)
        
        category = purpose_category[index]

        total_user = data['purpose_category'][data['purpose_category'] == purpose].count()
        user_wo_debt = data['purpose_category'][(data['purpose_category'] == purpose) 
                                                     & (data['debt'] == 0)].count()
        user_w_debt = data['purpose_category'][(data['purpose_category'] == purpose) 
                                                    & (data['debt'] == 1)].count()

        print(f'Кол-во кредитов на "{purpose}", имевших задолжности:', user_w_debt,
                                                    'шт.', '-', round(user_w_debt / total_user * 100,2), '%')

In [71]:
function_purpose_category()

Кол-во кредитов на "операции с автомобилем", имевших задолжности: 364 шт. - 9.39 %
Кол-во кредитов на "операции с недвижимостью", имевших задолжности: 712 шт. - 7.33 %
Кол-во кредитов на "получение образования", имевших задолжности: 331 шт. - 9.25 %
Кол-во кредитов на "проведение свадьбы", имевших задолжности: 156 шт. - 7.47 %


##### Вывод 4: Исходя из полученных результатов, можно сделать следующий вывод - кредиты на автомобиль это самая проблемная категория. Далее идет получение образования. При выдачи кредитов на данные нужды, нужно более тщательнее проводить проверки кредиторов.

## Общий вывод: По результатам исследования были получены результаты, которые дают информацию , о том, как можно повысить безопасность выдачи кредитов.

 <div class="alert alert-info">
    <h2> Комментарий студента</h2>
    Общий вывод.
С точки зрения влияния кол-ва детей в семье на вероятность задолженности по кредиту, выделяются две категории с одним и двумя детьми. Процент проблемных заемщиков с одним и двумя детьми составляет соответственно - 9,29% и 9,72%. Самые надежные категории, это семьи с тремя детьми и бездетные, процент должников ~7,50%.
Чуть больше 10% — это количество кредитов, имевших задолженности по выплатам у людей, находящихся в статусе "не в браке". Это число вполне поддается объяснению, вероятнее всего, это люди молодого возраста, которые менее ответственны, чем люди, находящиеся в браке. Семейные люди - самая большая категория среди представленных, имеющая процент по задолженностям 7,59%, самая надежная. Так же низкий процент среди категорий "вдова", "в разводе", но эти категории малочисленны и поэтому не делаем особый упор на эти данные.
Влияние уровня дохода кредитора не выявило закономерности в результатах исследования. В виду того, что категории "A, D, E" (380 человек, три категории вместе взятые) достаточно малочисленны по сравнению с категорией "С" (1183 человека), можно лишь выделить проблемную категорию с уровнем дохода - категория "С" - 8,54%.
Исследование по целям кредита выявило следующую закономерность, люди берущие кредит на машину или обучение , чаще имеют проблемы с выплатой в срок - 9,39% и 9,25% соответственно. А кредиты на недвижимость и свадьбу имеют лучший процент невозврата в срок - 7,33% и 7,47% соответственно.
    </div>